In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# Load the fashion-mnist pre-shuffled train data and test data


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

In [ ]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [ ]:
(x_train, x_valid) = x_train[5000:], x_train[:5000] 
(y_train, y_valid) = y_train[5000:], y_train[:5000]

# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
x_train = x_train.reshape(x_train.shape[0], w, h, 1)
x_valid = x_valid.reshape(x_valid.shape[0], w, h, 1)
x_test = x_test.reshape(x_test.shape[0], w, h, 1)

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_valid = tf.keras.utils.to_categorical(y_valid, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Print training set shape
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

# Print the number of training, validation, and test datasets
print('Train set', x_train.shape[0])
print('Validation set', x_valid.shape[0])
print('Test set', x_test.shape[0])

In [ ]:
# Custom Activation
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
from keras import backend as K

def Generic_Para(x):
    return 0.65*K.pow(x,2) + 0.5*x

def SBAF_Para(x):
    return 1 - x + K.pow(x, 2)

k = 1.25
def A_ReLU_Para(x):
    global k
    return k * K.pow(x, 2)

def Abs_Func(x):
    return K.abs(x)

#Not Working
def A_ReLU(x):
  k = 1.06
  n = 1.1
  return K.max(0, k*K.pow(x,n))

def Leaky_A_ReLU(x):
  a = 0.01
  k = 1.06
  n = 1.1
  if x>=0:
    return k*K.pow(x,n)
  else:
    return a*k*K.pow(K.abs(x),n)
  
get_custom_objects().update({
    'Generic_Para': Activation(Generic_Para),
    'SBAF_Para': Activation(SBAF_Para),
    'A_ReLU_Para': Activation(A_ReLU_Para),
    'Abs_Func': Activation(Abs_Func),
    'A_ReLU': Activation(A_ReLU),
    'Leaky_A_ReLU': Activation(Leaky_A_ReLU)
    })


In [ ]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
# Must define the input shape in the first layer of the neural network
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
# model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='Generic_Para'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
# Take a look at the model summary
model.summary()


In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])


In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='relu.model.weights.best.hdf5', verbose = 1, save_best_only=True)
model.fit(x_train,
         y_train,
         batch_size=64,
         epochs=10,
         validation_data=(x_valid, y_valid),
         callbacks=[checkpointer])


In [ ]:
# model.load_weights('model.weights.best.hdf5')
score = model.evaluate(x_test, y_test, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', round(score[1], 5))
print('\n', 'Test Loss:', round(score[0], 5))